#**Aula 2 de Labortório de Aprendizado de Máquina**
#Regressão Linear e Classificação Baseada em Distância



##Imports das bibliotecas principais

In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn import neighbors
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import feature_selection
import matplotlib.pyplot as plt

##Regressão Linear

**1) Obtenha por regressão linear uma função que relaciona a variável x com a variável y. Verifique o efeito de padronizar por Z-score (média zero e desvio padrão unitário) x e y (com os dados de treinamento)**

In [ ]:
np.random.seed(0)

x = np.arange(0,10, 0.05)
y = 2 + 3*x + np.random.normal(size=(200))
plt.plot(x,y)
plt.xlabel('x')
plt.ylabel('y')

x_train = x[0:100]
y_train = y[0:100]

x_test = x[100:]
y_test = y[100:]

#Z-score
#mean_x = np.mean(x_train)
#std_x = np.std(x_train)
#mean_y = np.mean(y_train)
#std_y = np.std(y_train)

#x_train = (x_train - mean_x)/std_x
#x_test = (x_test - mean_x)/std_x
#y_train = (y_train - mean_y)/std_y
#y_test = (y_test - mean_y)/std_y

X = np.vstack((np.ones(100),x_train)).T
w = np.linalg.inv(X.T @ X) @ X.T @ y_train
print(f'coeficientes método da pseudo inversa: {w}')

y_est = np.vstack((np.ones(100),x_test)).T @ w

RMSE = np.sqrt(np.sum((y_est - y_test)**2)/y_test.size)
print(f'RMSE: {"{:.4f}".format(RMSE)}')

x_train = np.array([x_train]).T
y_train = np.array([y_train]).T

reg = linear_model.LinearRegression().fit(x_train, y_train)
print(f'coeficientes método do sklearn: {np.vstack([reg.intercept_, reg.coef_])}')

x_test = np.array([x_test]).T
y_test = np.array([y_test]).T
RMSE = np.sqrt(np.sum((reg.predict(x_test) - y_test)**2)/y_test.size)
print(f'RMSE do sklearn: {"{:.4f}".format(RMSE)}')

**2) Estime os valores das casas da base de dados california_housing por regressão linear**

In [ ]:
raw_data = datasets.fetch_california_housing()
print(raw_data.DESCR)

data = pd.DataFrame(np.c_[raw_data["data"],raw_data["target"]], columns=list(raw_data["feature_names"]) + ["target"])
display(data.head())
data.describe()

In [ ]:
features_of_interest = ["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup", "Latitude", "Longitude"]
X_train = data.iloc[0:15000][features_of_interest]
X_test = data.iloc[15000:][features_of_interest]

y_train = data.iloc[0:15000]["target"]
y_test = data.iloc[15000:]["target"]

reg = linear_model.LinearRegression().fit(X_train, y_train)
print(f'coeficientes método do sklearn: {np.hstack([reg.intercept_, reg.coef_])}')
RMSE = np.sqrt(np.sum((reg.predict(X_test) - y_test)**2)/y_test.shape[0])
print(f'RMSE do sklearn: {"{:.4f}".format(RMSE)}')

#reg = linear_model.Lasso().fit(X_train, y_train)
#print(f'coeficientes método do sklearn: {np.hstack([reg.intercept_, reg.coef_])}')
#RMSE = np.sqrt(np.sum((reg.predict(X_test) - y_test)**2)/y_test.shape[0])
#print(f'RMSE do sklearn: {"{:.4f}".format(RMSE)}')

**3) Use o kNN para classificar a base de dados iris. Use métodos de hold-out e k-fold para avaliar o resultado. Use o método de Grid Search para encontrar o melhor valor de k**

In [ ]:
raw_data = datasets.load_iris()
print(raw_data.DESCR)

data = pd.DataFrame(np.c_[raw_data["data"],raw_data["target"]], columns=list(raw_data["feature_names"]) + ["target"])
display(data.head())
data.describe()

In [ ]:

X = data.drop(columns=["target"])
y = data["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify = y)

model = neighbors.KNeighborsClassifier(n_neighbors = 3)
model.fit(X_train,y_train)
model.predict(X_test)
acc = model.score(X_test, y_test)
print(f"Acurácia: {acc*100}%")

#k-fold
#cv_scores = cross_val_score(model, X, y, cv=5, scoring = "accuracy")
#print(cv_scores)
#print("cv_scores médio:{}".format(np.mean(cv_scores)))

#Grid Search
#model = neighbors.KNeighborsClassifier()
#param_grid = {"n_neighbors": np.arange(1, 10)}
#knn_gscv = GridSearchCV(model, param_grid, cv=5)
#knn_gscv.fit(X_train, y_train)
#print(f"Melhor valor de k: {knn_gscv.best_params_} e melhor resultado: {knn_gscv.best_score_}")

#for dicts in knn_gscv.best_params_:
#  k = knn_gscv.best_params_[dicts]

#model = neighbors.KNeighborsClassifier(n_neighbors = k)
#model.fit(X_train,y_train)
#model.predict(X_test)
#acc = model.score(X_test, y_test)
#print(f"Acurácia: {acc*100}%")

**4) Realize a classificação da base de dados Wine com o kNN e o Rocchio. Avalie o uso de Z-score e do SFS para seleção de features**

In [ ]:
raw_data = datasets.load_wine()
print(raw_data.DESCR)

data = pd.DataFrame(np.c_[raw_data["data"],raw_data["target"]], columns=list(raw_data["feature_names"]) + ["target"])
display(data.head())
data.describe()

In [ ]:
X = data.drop(columns=["target"])
y = data["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify = y)


#Z-score
#mean_X = np.mean(X_train,axis=0)
#std_X = np.std(X_train,axis=0)
#X_train = (X_train - mean_X)/std_X
#X_test = (X_test - mean_X)/std_X


#SFS
#model = neighbors.KNeighborsClassifier(n_neighbors = 3)
#sfs = feature_selection.SequentialFeatureSelector(model,n_features_to_select=3)
#sfs.fit(X_train, y_train)
#print(sfs.get_support())
#X_train = sfs.transform(X_train)
#X_test = sfs.transform(X_test)


model = neighbors.KNeighborsClassifier(n_neighbors = 3)
model.fit(X_train,y_train)
model.predict(X_test)
acc = model.score(X_test, y_test)
print(f"Acurácia do kNN: {acc*100}%")


#Rocchio ou Classificador Baseado em Centroide
#classes = y_train.unique()
#X_train_prot = np.zeros([classes.size,X_train.shape[1]])
#y_train_prot = np.zeros([classes.size,1])
#for classe in range(0,classes.size):
#  X_train_prot[classe] = np.mean(X_train[y_train==classe],axis=0)
#  y_train_prot[classe] = classe
#y_train_prot = y_train_prot.ravel()

#print(X_train_prot)

#model = neighbors.KNeighborsClassifier(n_neighbors = 1)
#model.fit(X_train_prot,y_train_prot)
#model.predict(X_test)
#acc = model.score(X_test, y_test)
#print(f"Acurácia do Rocchio implementado: {acc*100}%")

#model = neighbors.NearestCentroid()
#model.fit(X_train,y_train)
##print(model.centroids_)
#model.predict(X_test)
#acc = model.score(X_test, y_test)
#print(f"Acurácia do Rocchio do sklearn: {acc*100}%")

